## The new intake GUI - built with panel

It is often useful to explore catalog data using a Graphical User Interface. The previous iteration of the intake GUI was built in ipywidgets - which limited you to use from within the notebook. The new version is implemented in [panel](https://panel.pyviz.org) which means that in addition to rendering in the notebook - it also can easily be served and deployed as a tornado application. 


**NOTE:** At the moment this work depends on development version of panel and bokeh


The intake GUI is composed of various sub-panels that can be opened and closed from the control pane on the left. Try clicking the + button to get started.

In [ ]:
import intake
intake.output_notebook()  # I think this is only needed in jupyter notebook (not in lab)

In [ ]:
intake.gui

Try loading remote catalog available at: https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml

Get the selected item from the gui:

In [ ]:
intake.gui.item

This item is a regular intake catalog entry object that can be used in the regular ways. 

In [ ]:
dataArray = intake.gui.item.to_dask()
dataArray

You can also look at the GUI just for that entry:

In [ ]:
intake.gui.item.gui

OR you can get just one sub-panel from the gui and work with that directly:

In [ ]:
intake.gui.description

In [ ]:
import param
import holoviews as hv
import panel as pn

In [ ]:
land_dict = {land:i for i,land in enumerate(list(dataArray.landuse.values))}

class LandExplorer(param.Parameterized):
    
    cmap = param.ObjectSelector(default ='Blues', objects=hv.plotting.list_cmaps())
    land = param.ObjectSelector(default='agricultural', objects=list(dataArray.landuse.values))
    file =  param.ObjectSelector(default=0, objects = list(range(100)))
    
    
    @param.depends('cmap','land','file')
    def get_image_from_file(self):
        cmap = self.cmap
        land = self.land
        file = self.file
        index = land_dict[land]
        return hv.Image(dataArray.values[file][index]).options(width=500, height=500,cmap = cmap)

In [ ]:
l = LandExplorer()

dmap = pn.Row(hv.DynamicMap(l.get_image_from_file),l.param)

dashboard = pn.Column(
    pn.Column('# <pre> Land Explorer',
              pn.Spacer(height=50)),
    pn.Row(
        pn.Row(dmap[0],
               pn.Spacer(width = 50),
               pn.Column(
                   pn.Spacer(height = 60),
                   dmap[1][1],
                   pn.Spacer(height = 30),
                   dmap[1][2],
                   pn.Spacer(height = 30),
                   dmap[1][3])
              )
        )
)

dashboard

You can also get a gui directly off a cat like:

In [ ]:
cat = intake.open_catalog('./us_crime/us_crime.yaml')

In [ ]:
cat.gui

And do regular intake-y stuff on it.

In [ ]:
cat.gui.item

In [ ]:
us_crime = cat.gui.item

us_crime.plot.bivariate('Burglary rate', 'Property crime rate', legend=False, width=500, height=400) * \
us_crime.plot.scatter(  'Burglary rate', 'Property crime rate', color='black', size=15, legend=False) +\
us_crime.plot.table(['Burglary rate', 'Property crime rate'], width=350, height=350)